In [ ]:
import keras

from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.layers import Input, LSTM, Embedding, Dense, Dropout
from keras.models import Model, Sequential

# First, let's define a vision model using a Sequential model.
# This model will encode an image into a vector.
vision_model = Sequential()
# vision_model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
vision_model.add(Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=(28, 28, 1)))
vision_model.add(MaxPooling2D((2, 2)))
vision_model.add(Conv2D(16, (3, 3), activation='relu'))
vision_model.add(MaxPooling2D((2, 2)))


vision_model.add(Flatten())
vision_model.add(Dense(128, activation='relu'))
vision_model.add(Dense(10, activation='softmax'))

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [ ]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

x_train = x_train.astype('float32')   # int型をfloat32型に変換
x_test = x_test.astype('float32')
x_train /= 255                        # [0-255]の値を[0.0-1.0]に変換
x_test /= 255

# convert class vectors to binary class matrices
num_classes = 10

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

60000 train samples
60000 test samples


In [ ]:
print(y_train[0])

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [ ]:
vision_model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
vision_model.fit(x_train, y_train,  epochs = 5, batch_size = 128 )  # starts training


Epoch 1/5
60000/60000 [==============================] - 6s 92us/step - loss: 0.2592 - accuracy: 0.9215
Epoch 2/5
60000/60000 [==============================] - 2s 38us/step - loss: 0.0702 - accuracy: 0.9784
Epoch 3/5
60000/60000 [==============================] - 2s 38us/step - loss: 0.0497 - accuracy: 0.9841
Epoch 4/5
60000/60000 [==============================] - 2s 37us/step - loss: 0.0381 - accuracy: 0.9883
Epoch 5/5
60000/60000 [==============================] - 2s 38us/step - loss: 0.0301 - accuracy: 0.9909


In [ ]:
vision_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 16)        2320      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 16)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               73856     
_________________________________________________________________
dense_2 (Dense)              (None, 10)               

In [ ]:
model_mid = Model(inputs=vision_model.input,
outputs=vision_model.get_layer('flatten_1').output)

In [ ]:
import numpy as np

model_mid.predict(np.array([x_test[0]]))

array([[0.00000000e+00, 8.11709315e-02, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        5.02298363e-02, 1.45997167e-01, 0.00000000e+00, 0.00000000e+00,
        3.72431964e-01, 1.46089524e-01, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 7.11894870e-01,
        0.00000000e+00, 5.70673607e-02, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 9.95799482e-01, 1.68966427e-01, 3.07128727e-01,
        2.37309054e-01, 1.49258912e-01, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 4.82612908e-01,
        0.00000000e+00, 8.92507583e-02, 5.11087835e-01, 0.00000000e+00,
        0.00000000e+00, 2.06496215e+00, 1.72323301e-01, 0.00000000e+00,
        7.08393008e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 2.83351421e-01, 3.201883

In [ ]:
def gen_featurevec(x, model):
    fvec = model.predict(x)
    
    return fvec

feature_vec = gen_featurevec(x_train, model)

In [ ]:
print(feature_vec.shape)

(60000, 576)


In [ ]:
print(feature_vec)

[[0.         0.08117093 0.         ... 0.00983861 0.         0.        ]
 [0.         0.06641662 0.         ... 0.         0.         0.        ]
 [0.43061486 0.11544582 0.         ... 0.00099546 0.         0.        ]
 ...
 [0.         0.06641662 0.         ... 0.         0.         0.        ]
 [0.         0.06641662 0.         ... 0.         0.         0.        ]
 [0.         0.06641662 0.         ... 0.         0.         0.        ]]


In [ ]:
sequence_size = 2
output_size = 19

def sampling_data(x,y, sampling_size=50000):
    x_T = []
    y_T = []
    batch_size = sampling_size
    for _ in range(sequence_size):
        x_batch=[]
        y_batch=[]
        for i in range(batch_size):
            step = np.random.randint(0, len(x))
            # x batch
            x_step = x[step]
            # x_step = np.reshape(x_step, [img_size, img_size, 1])
            x_batch.append(x_step)
            # y batch
            y_step = int(y[step])
            y_batch.append(y_step)

        # x list of sequence
        x_T.append(x_batch)
        # y list of sequence
        y_T.append(y_batch)

    #print(np.asarray(y_T[0]))
    #print(np.asarray(y_T[1]))
    y_T=np.asarray(y_T[0])-np.asarray(y_T[1])

    #print(y_T)
    #one hot encording
    y_T_onehot=[]
    for i in range(batch_size):
        y_onehot = np.zeros(output_size)
        y_onehot[y_T[i]+9]=1
        y_T_onehot.append(y_onehot)

    return x_T,y_T,y_T_onehot

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_T,y_T,y_T_onehot = sampling_data(feature_vec, y_train)

In [ ]:
y_T_onehot = np.array(y_T_onehot)

In [ ]:
x_T = np.array(x_T)

In [ ]:
print(x_T.shape)
x_T = np.transpose(x_T, (1,0, 2))
print(np.shape(x_T))

(2, 50000, 576)
(50000, 2, 576)


In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers.recurrent import LSTM
from keras.optimizers import Adam

seq_length = 2
n_in = 576
n_hidden = 128
n_out = 19

model=Sequential()
model.add(LSTM(units=n_hidden, input_shape=(seq_length, n_in)))
model.add(Dense(units=n_out))
model.add(Activation('softmax'))

optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


import numpy as np

#query = [[[1,0,0,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0,0,0]],[[0,1,0,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0,0,0]]]
#answer = [[0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]

#query = np.array(query)
#answer = np.array(answer)

batch_size=20

query = x_T
answer = y_T_onehot

model.fit(query, answer,
          batch_size=batch_size,
          epochs=10,
          validation_split=0.1,
          )
 


Train on 45000 samples, validate on 5000 samples
Epoch 1/10
45000/45000 [==============================] - 13s 293us/step - loss: 0.6695 - accuracy: 0.8102 - val_loss: 0.1990 - val_accuracy: 0.9458
Epoch 2/10
45000/45000 [==============================] - 13s 286us/step - loss: 0.1402 - accuracy: 0.9615 - val_loss: 0.1327 - val_accuracy: 0.9622
Epoch 3/10
45000/45000 [==============================] - 13s 286us/step - loss: 0.0831 - accuracy: 0.9769 - val_loss: 0.1317 - val_accuracy: 0.9586
Epoch 4/10
45000/45000 [==============================] - 13s 288us/step - loss: 0.0508 - accuracy: 0.9862 - val_loss: 0.1248 - val_accuracy: 0.9612
Epoch 5/10
45000/45000 [==============================] - 13s 291us/step - loss: 0.0354 - accuracy: 0.9906 - val_loss: 0.1275 - val_accuracy: 0.9628
Epoch 6/10
45000/45000 [==============================] - 13s 290us/step - loss: 0.0235 - accuracy: 0.9941 - val_loss: 0.1211 - val_accuracy: 0.9660
Epoch 7/10
45000/45000 [==============================] -

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_test = x_test.astype('float32')
x_test /= 255

feature_vec_test = gen_featurevec(x_test, model_mid)

In [ ]:
#test_query = [[[1,0,0,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0,0,0]]]
#test_query = np.array(test_query)

test_query = [feature_vec_test[900], feature_vec_test[902]]
test_query = np.array([test_query])

print(feature_vec_test[900])

print(test_query.shape)

predicted = model.predict(test_query)

print(y_test[909])
print(y_test[142])

print(np.argmax(predicted)-9)

[0.00000000e+00 6.64166212e-02 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 6.64166212e-02 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 9.39225852e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 3.55341077e-01
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 5.49923301e-01 6.55128181e-01 0.00000000e+00 1.34089971e+00
 1.12938833e+00 0.00000000e+00 7.17484772e-01 3.83273140e-02
 0.00000000e+00 1.43862180e-02 1.74396545e-01 1.32290137e+00
 6.84242964e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
 8.01531732e-01 9.27671134e-01 1.17458010e+00 5.89617372e-01
 0.00000000e+00 6.641662

In [ ]:
query = [[[1,0,0,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0,0,0]],[[0,1,0,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0,0,0]]]
answer = [[0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]

query = np.array(query)
answer = np.array(answer)

In [ ]:
print(query.shape)

(2, 2, 10)
